In [1]:
from otit_swt_mapper import Mapping, ResolveIRI, MintingOptions, to_graph
import polars as pl
from math import floor

# Templates

In [2]:
instance_mapping = """
@prefix tpl:<https://github.com/magbak/otit_swt/templates#>.
@prefix rds:<https://github.com/magbak/otit_swt/rds_power#>.
@prefix otit:<https://github.com/magbak/otit_swt#>.
@prefix rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs:<http://www.w3.org/2000/01/rdf-schema#>.

tpl:Site [?SiteIRI, ?SiteName] :: {
    ottr:Triple(?SiteIRI, rdfs:label, ?SiteName),
    ottr:Triple(?SiteIRI, rdf:type, rds:Site)
    } .
    
tpl:FunctionalAspect [?SourceIRI, ?TargetIRI, ?TargetAspectNodeIRI, ?Label] :: {
    ottr:Triple(?SourceIRI, rds:hasFunctionalAspect, ?TargetAspectNodeIRI),
    ottr:Triple(?TargetIRI, rds:hasFunctionalAspectNode, ?TargetAspectNodeIRI),
    ottr:Triple(?TargetAspectNodeIRI, rdfs:label, ?Label)
} .

tpl:RDSSystem [?SystemIRI, ?RDSType, ?Label] :: {
    ottr:Triple(?SystemIRI, rdf:type, ?RDSType),
    ottr:Triple(?SystemIRI, rdfs:label, ?Label)
} .

tpl:StaticProperty [?ParentIRI, ?ValueNodeIRI, ?Label, ?Value] :: {
    ottr:Triple(?ParentIRI, otit:hasStaticProperty, ?ValueNodeIRI),
    ottr:Triple(?ValueNodeIRI, rdfs:label, ?Label),
    ottr:Triple(?ValueNodeIRI, otit:hasStaticValue, ?Value)
} .

tpl:Timeseries [?ParentIRI, ?TimeseriesNodeIRI, ?Label, ?ExternalId, ?Datatype] :: {
    ottr:Triple(?ParentIRI, otit:hasTimeseries, ?TimeseriesNodeIRI),
    ottr:Triple(?TimeseriesNodeIRI, otit:hasExternalId, ?ExternalId),
    ottr:Triple(?TimeseriesNodeIRI, otit:hasDatatype, ?Datatype),
    ottr:Triple(?TimeseriesNodeIRI, rdfs:label, ?Label)
} .
"""

In [3]:
n = 1_000

# Mapping object

In [4]:
mapping = Mapping([instance_mapping])

In [5]:
# Used as a prefix
wpex = "https://github.com/magbak/otit_swt/windpower_example#"

In [6]:
sites = pl.DataFrame({"Key":["SiteA", "SiteB"], "SiteName": ["Wind Mountain", "Gale Valley"]})
sites_mint_iris = {"SiteIRI":MintingOptions(prefix=wpex+"Site", numbering_suffix_start=0)}
mapping.expand("tpl:Site", sites, mint_iris=sites_mint_iris)

Key,SiteIRI
str,str
"""SiteA""","""https://github..."
"""SiteB""","""https://github..."


# Wind turbines

In [7]:
wind_turbine_keys = ["WindTurbine" + str(i) for i in range(1,n+1)]
wind_turbines = pl.DataFrame(
    {
    "Key":wind_turbine_keys,
    "RDSType": ["A"]*n,
    "Label": ["Wind turbine " + str(i) for i in range(1,n+1)]
    })
wind_turbines_mint_iris = {
    "SystemIRI":MintingOptions(
        prefix=wpex + "WindTurbine", numbering_suffix_start=1),
    }
mapping.expand("tpl:RDSSystem", wind_turbines, 
               mint_iris=wind_turbines_mint_iris)

Key,SystemIRI
str,str
"""WindTurbine1""","""https://github..."
"""WindTurbine2""","""https://github..."
"""WindTurbine3""","""https://github..."
"""WindTurbine4""","""https://github..."
"""WindTurbine5""","""https://github..."
"""WindTurbine6""","""https://github..."
"""WindTurbine7""","""https://github..."
"""WindTurbine8""","""https://github..."
"""WindTurbine9""","""https://github..."


In [8]:
operating_external_ids = ["oper" + str(i) for i in range(1,n+1)]
operating = pl.DataFrame ({
    "Key": [k + "_operating" for k in wind_turbine_keys],
    "Label": ["Operating"]*n,
    "ParentIRIKey" :wind_turbine_keys,
    "ExternalId": operating_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#boolean"]*n
})
operating_mint_iris = {
    "TimeseriesNodeIRI": MintingOptions(prefix=wpex + "Operating",
                                        numbering_suffix_start=1)
}
operating_resolve_iris = {
    "ParentIRI": ResolveIRI(key_column="ParentIRIKey", 
                           template="tpl:RDSSystem",
                           argument="SystemIRI")
}
mapping.expand("tpl:Timeseries", operating, 
               mint_iris=operating_mint_iris,
              resolve_iris=operating_resolve_iris)

Key,TimeseriesNodeIRI
str,str
"""WindTurbine1_o...","""https://github..."
"""WindTurbine2_o...","""https://github..."
"""WindTurbine3_o...","""https://github..."
"""WindTurbine4_o...","""https://github..."
"""WindTurbine5_o...","""https://github..."
"""WindTurbine6_o...","""https://github..."
"""WindTurbine7_o...","""https://github..."
"""WindTurbine8_o...","""https://github..."
"""WindTurbine9_o...","""https://github..."


In [9]:
maximum_power_values = [[5_000_000,10_000_000,15_000_000][i%3] for i in range(1,n+1)]
maximum_power = pl.DataFrame ({
    "Key": [k + "_maxpower" for k in wind_turbine_keys],
    "Label": ["Maximum Power"] * n,
    "Value": maximum_power_values,
    "ParentIRIKey": wind_turbine_keys,
})
maximum_power_mint_iris = {
    "ValueNodeIRI":MintingOptions(
        prefix=wpex + "WindTurbineMaximumPower", numbering_suffix_start=1),
    }
maximum_power_resolve_iris = {
    "ParentIRI": ResolveIRI(key_column="ParentIRIKey",
                           template="tpl:RDSSystem",
                           argument="SystemIRI")
}
mapping.expand("tpl:StaticProperty", 
               df=maximum_power,
               mint_iris=maximum_power_mint_iris,
               resolve_iris=maximum_power_resolve_iris)

Key,ValueNodeIRI
str,str
"""WindTurbine1_m...","""https://github..."
"""WindTurbine2_m...","""https://github..."
"""WindTurbine3_m...","""https://github..."
"""WindTurbine4_m...","""https://github..."
"""WindTurbine5_m...","""https://github..."
"""WindTurbine6_m...","""https://github..."
"""WindTurbine7_m...","""https://github..."
"""WindTurbine8_m...","""https://github..."
"""WindTurbine9_m...","""https://github..."


In [10]:
site_has_wind_turbine = pl.DataFrame({
    "SourceIRIKey":["SiteA" if i%2 == 0 else "SiteB" for i in range(1,n+1)],
    "TargetIRIKey":wind_turbine_keys,
    "Label": ["A" + str(int(floor(i/2))+i%2) for i in range(1,n+1)],
})
site_has_wind_turbine["Key"] = site_has_wind_turbine["SourceIRIKey"] + "_" + site_has_wind_turbine["TargetIRIKey"]
site_has_wind_turbine_mint_iris = {
    "TargetAspectNodeIRI": MintingOptions(
        prefix= wpex + "WindTurbineFunctionalAspect", 
        numbering_suffix_start=1)}
site_has_wind_turbine_resolve_iris = {
    "SourceIRI":ResolveIRI(key_column="SourceIRIKey",
                           template="tpl:Site",
                           argument="SiteIRI"
                          ),
    "TargetIRI":ResolveIRI(key_column="TargetIRIKey",
                           template="tpl:RDSSystem",
                           argument="SystemIRI"
                          ),
}
mapping.expand("tpl:FunctionalAspect", site_has_wind_turbine, 
               mint_iris=site_has_wind_turbine_mint_iris, 
               resolve_iris=site_has_wind_turbine_resolve_iris)

Key,TargetAspectNodeIRI
str,str
"""SiteB_WindTurb...","""https://github..."
"""SiteA_WindTurb...","""https://github..."
"""SiteB_WindTurb...","""https://github..."
"""SiteA_WindTurb...","""https://github..."
"""SiteB_WindTurb...","""https://github..."
"""SiteA_WindTurb...","""https://github..."
"""SiteB_WindTurb...","""https://github..."
"""SiteA_WindTurb...","""https://github..."
"""SiteB_WindTurb...","""https://github..."


# Generator systems

In [11]:
generator_system_keys = ["GeneratorSystem" + str(i) for i in range(1,n+1)]
generator_systems = pl.DataFrame(
    {
    "Key":generator_system_keys,
    "Label": ["Generator system"]*n,
    "RDSType": ["RA"]*n
    })
generator_systems_mint_iris = {
    "SystemIRI":MintingOptions(
        prefix=wpex + "GeneratorSystem", numbering_suffix_start=1)}
mapping.expand("tpl:RDSSystem", generator_systems, 
               mint_iris=generator_systems_mint_iris)

Key,SystemIRI
str,str
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."


In [12]:
wind_turbine_has_generator_system = pl.DataFrame ({
    "SourceIRIKey": wind_turbine_keys,
    "TargetIRIKey": generator_system_keys,
    "Label": ["RA1" for i in range(1,n+1)],
})
wind_turbine_has_generator_system["Key"] = wind_turbine_has_generator_system["SourceIRIKey"] + "_" + wind_turbine_has_generator_system["TargetIRIKey"]
wind_turbine_has_generator_system_mint_iris = {
    "TargetAspectNodeIRI": MintingOptions(
        prefix= wpex + "GeneratorFunctionalAspect", 
        numbering_suffix_start=1)}
rdssystem_to_rdssystem_resolve_iris = {
    "SourceIRI":ResolveIRI(key_column="SourceIRIKey",
                           template="tpl:RDSSystem",
                           argument="SystemIRI"
                          ),
    "TargetIRI":ResolveIRI(key_column="TargetIRIKey",
                           template="tpl:RDSSystem",
                           argument="SystemIRI"
                          ),
}
mapping.expand("tpl:FunctionalAspect", wind_turbine_has_generator_system, 
               mint_iris=wind_turbine_has_generator_system_mint_iris, 
               resolve_iris=rdssystem_to_rdssystem_resolve_iris)

Key,TargetAspectNodeIRI
str,str
"""WindTurbine1_G...","""https://github..."
"""WindTurbine2_G...","""https://github..."
"""WindTurbine3_G...","""https://github..."
"""WindTurbine4_G...","""https://github..."
"""WindTurbine5_G...","""https://github..."
"""WindTurbine6_G...","""https://github..."
"""WindTurbine7_G...","""https://github..."
"""WindTurbine8_G...","""https://github..."
"""WindTurbine9_G...","""https://github..."


# Generator

In [13]:
generator_keys = ["Generator" + str(i) for i in range(1,n+1)]
generators = pl.DataFrame(
    {
    "Key": generator_keys,
    "RDSType": ["GAA"]*n,
    "Label" : ["Generator"]*n
    })
generators_mint_iris = {
    "SystemIRI":MintingOptions(
        prefix=wpex + "Generator", numbering_suffix_start=1)}
mapping.expand("tpl:RDSSystem", generators, 
               mint_iris=generators_mint_iris)

Key,SystemIRI
str,str
"""Generator1""","""https://github..."
"""Generator2""","""https://github..."
"""Generator3""","""https://github..."
"""Generator4""","""https://github..."
"""Generator5""","""https://github..."
"""Generator6""","""https://github..."
"""Generator7""","""https://github..."
"""Generator8""","""https://github..."
"""Generator9""","""https://github..."


In [14]:
energy_production_external_ids = ["ep" + str(i) for i in range(1,n+1)]
energy_production = pl.DataFrame ({
    "Key": [k + "_production" for k in generator_keys],
    "Label": ["Production"]*n,
    "ParentIRIKey" :generator_keys,
    "ExternalId": energy_production_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
energy_production_mint_iris = {
    "TimeseriesNodeIRI": MintingOptions(prefix=wpex + "EnergyProduction",
                                        numbering_suffix_start=1)
}
energy_production_resolve_iris = {
    "ParentIRI": ResolveIRI(key_column="ParentIRIKey", 
                           template="tpl:RDSSystem",
                           argument="SystemIRI")
}
mapping.expand("tpl:Timeseries", energy_production, 
               mint_iris=energy_production_mint_iris,
              resolve_iris=energy_production_resolve_iris)

Key,TimeseriesNodeIRI
str,str
"""Generator1_pro...","""https://github..."
"""Generator2_pro...","""https://github..."
"""Generator3_pro...","""https://github..."
"""Generator4_pro...","""https://github..."
"""Generator5_pro...","""https://github..."
"""Generator6_pro...","""https://github..."
"""Generator7_pro...","""https://github..."
"""Generator8_pro...","""https://github..."
"""Generator9_pro...","""https://github..."


In [15]:
generator_system_has_generator = pl.DataFrame ({
    "SourceIRIKey": generator_system_keys,
    "TargetIRIKey": generator_keys,
    "Label": ["GAA1" for i in range(1,n+1)],
})
generator_system_has_generator["Key"] = generator_system_has_generator["SourceIRIKey"] + "_" + generator_system_has_generator["TargetIRIKey"]
generator_system_has_generator_mint_iris = {
    "TargetAspectNodeIRI": MintingOptions(
        prefix= wpex + "GeneratorFunctionalAspect", 
        numbering_suffix_start=1)}
mapping.expand("tpl:FunctionalAspect", generator_system_has_generator, 
               mint_iris=generator_system_has_generator_mint_iris, 
               resolve_iris=rdssystem_to_rdssystem_resolve_iris)

Key,TargetAspectNodeIRI
str,str
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."
"""GeneratorSyste...","""https://github..."


# Weather measuring system

In [16]:
wms_keys = ["WeatherMeasuringSystem" + str(i) for i in range(1,n+1)]
wms = pl.DataFrame(
    {
    "Key":wms_keys,
    "RDSType": ["LE"]*n,
    "Label": ["Weather Measuring System"]*n
    })
wms_mint_iris = {
    "SystemIRI":MintingOptions(
        prefix="https://github.com/magbak/otit_swt/windpower_example#WeatherMeasuringSystem", 
        numbering_suffix_start=1)
    }
mapping.expand("tpl:RDSSystem", wms, 
               mint_iris=wms_mint_iris)

Key,SystemIRI
str,str
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."


In [17]:
wind_speed_external_ids = ["wsp" + str(i) for i in range(1,n+1)]
wind_speed = pl.DataFrame ({
    "Key": [k + "_windspeed" for k in wms_keys],
    "Label": ["Windspeed"]*n,
    "ParentIRIKey" :wms_keys,
    "ExternalId": energy_production_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
wind_speed_mint_iris = {
    "TimeseriesNodeIRI": MintingOptions(prefix=wpex + "WindSpeed",
                                        numbering_suffix_start=1)
}
wind_speed_resolve_iris = {
    "ParentIRI": ResolveIRI(key_column="ParentIRIKey", 
                           template="tpl:RDSSystem",
                           argument="SystemIRI")
}
mapping.expand("tpl:Timeseries", wind_speed, 
               mint_iris=wind_speed_mint_iris,
              resolve_iris=wind_speed_resolve_iris)

Key,TimeseriesNodeIRI
str,str
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."


In [18]:
wind_direction_external_ids = ["wdir" + str(i) for i in range(1,n+1)]
wind_direction = pl.DataFrame ({
    "Key": [k + "_winddirection" for k in wms_keys],
    "Label": ["WindDirection"]*n,
    "ParentIRIKey" :wms_keys,
    "ExternalId": energy_production_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#double"]*n
})
wind_direction_mint_iris = {
    "TimeseriesNodeIRI": MintingOptions(prefix=wpex + "WindDirection",
                                        numbering_suffix_start=1)
}
wind_direction_resolve_iris = {
    "ParentIRI": ResolveIRI(key_column="ParentIRIKey", 
                           template="tpl:RDSSystem",
                           argument="SystemIRI")
}
mapping.expand("tpl:Timeseries", wind_direction, 
               mint_iris=wind_speed_mint_iris,
              resolve_iris=wind_speed_resolve_iris)

Key,TimeseriesNodeIRI
str,str
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."
"""WeatherMeasuri...","""https://github..."


In [19]:
energy_production_external_ids = ["ep" + str(i) for i in range(1,n+1)]
energy_production = pl.DataFrame ({
    "Key": [k + "_production" for k in generator_keys],
    "Label": ["Production"]*n,
    "ParentIRIKey" :wind_turbine_keys,
    "ExternalId": energy_production_external_ids,
    "Datatype": ["http://www.w3.org/2001/XMLSchema#integer"]*n
})
energy_production_mint_iris = {
    "TimeseriesNodeIRI": MintingOptions(prefix=wpex + "EnergyProduction",
                                        numbering_suffix_start=1)
}
energy_production_resolve_iris = {
    "ParentIRI": ResolveIRI(key_column="ParentIRIKey", 
                           template="tpl:RDSSystem",
                           argument="SystemIRI")
}
mapping.expand("tpl:Timeseries", energy_production, 
               mint_iris=energy_production_mint_iris,
              resolve_iris=energy_production_resolve_iris)

Key,TimeseriesNodeIRI
str,str
"""Generator1_pro...","""https://github..."
"""Generator2_pro...","""https://github..."
"""Generator3_pro...","""https://github..."
"""Generator4_pro...","""https://github..."
"""Generator5_pro...","""https://github..."
"""Generator6_pro...","""https://github..."
"""Generator7_pro...","""https://github..."
"""Generator8_pro...","""https://github..."
"""Generator9_pro...","""https://github..."


In [20]:
wind_turbine_has_wms = pl.DataFrame ({
    "SourceIRIKey": wind_turbine_keys,
    "TargetIRIKey": wms_keys,
    "Label": ["LE1" for i in range(1,n+1)],
})
wind_turbine_has_wms["Key"] = wind_turbine_has_wms["SourceIRIKey"] + "_" + wind_turbine_has_wms["TargetIRIKey"]
wind_turbine_has_wms_mint_iris = {
    "TargetAspectNodeIRI": MintingOptions(
        prefix= wpex + "WMSFunctionalAspect", 
        numbering_suffix_start=1)}
mapping.expand("tpl:FunctionalAspect", 
               df=wind_turbine_has_wms,
               mint_iris=wind_turbine_has_wms_mint_iris, 
               resolve_iris=rdssystem_to_rdssystem_resolve_iris
              )

Key,TargetAspectNodeIRI
str,str
"""WindTurbine1_W...","""https://github..."
"""WindTurbine2_W...","""https://github..."
"""WindTurbine3_W...","""https://github..."
"""WindTurbine4_W...","""https://github..."
"""WindTurbine5_W...","""https://github..."
"""WindTurbine6_W...","""https://github..."
"""WindTurbine7_W...","""https://github..."
"""WindTurbine8_W...","""https://github..."
"""WindTurbine9_W...","""https://github..."


In [21]:
triples = mapping.to_triples()

# Export to rdflib (slow)

In [22]:
gr = to_graph(mapping)

In [23]:
gr.serialize(destination="windpower.ttl")

<Graph identifier=Nc639b108973540619cb2ab762455c648 (<class 'rdflib.graph.Graph'>)>

# Write ntriples (fast)

In [24]:
mapping.write_ntriples("windpower.nt")